# 0. Short description 

In the first part of the lab you are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents, 
- then building an inverted index,
- and answering queries using this index.

Second part is devoted to spellchecking. You will implement some functions related to spellchecking.
- K-gram index
- weighted editorial distance
- Norvig spellchecker


# 1. [60] Building inverted index and answering queries

## 1.1. [10] Preprocessing

First, we need a unified approach to documents preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library or any other you know.

In [1]:
import nltk

class Preprocessor:
    
    def __init__(self):
        self.stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                      'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()
        self.tokenizer = nltk.tokenize.NLTKWordTokenizer()

    
    def tokenize(self, text):
        #TODO word tokenize text using nltk lib
        return self.tokenizer.tokenize(text)

    
    def stem(self, word, stemmer):
        #TODO stem word using provided stemmer
        return stemmer.stem(word)

    
    def is_apt_word(self, word):
        #TODO check if word is appropriate - not a stop word and isalpha, 
        # i.e consists of letters, not punctuation, numbers, dates
        return (not (word in self.stop_words)) and word.isalpha()

    
    def preprocess(self, text):
        #TODO combine all previous methods together: tokenize lowercased text 
        # and stem it, ignoring not appropriate words
        tokens = self.tokenizer.tokenize(text.lower())
        return list(map(lambda x: self.stem(x, self.ps), 
                        filter(lambda x: self.is_apt_word(x), tokens)))

### 1.1.1. Tests

In [10]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

## 1.2. [35] Crawling and Indexing

### 1.2.1. [10] Base classes

Here are some base classes you will need for writing an indexer. The code from the first lab's solution is given, still you need to change some of it, namely, the `parse` function (it is also possible to use your own implementation from the first homework, but make sure that it is correct). The reason is it always makes complete parsing, which we want to avoid when we only need links, for example, or a specific portion of text.

In [2]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os
from itertools import chain

class Document:

    def __init__(self, url):
        self.url = url

    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.type = response.headers['Content-Type']
                self.content = response.content
                return True
            else:
                return False
        except:
            return False

    def persist(self, path):
        if not os.path.exists(path):
            os.mkdir(path)
#         print(os.path.join(path, quote(self.url).replace('/', '_')))
        # Had problems with the original code
        with open(os.path.join(path, str(hash(self.url))), 'wb') as f:
            f.write(self.content)


class HtmlDocument(Document):

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        return href

    def parse(self, is_article=False):
        #TODO change this method   
        # Checking if it is html
        if 'html' in self.type:
            model = BeautifulSoup(self.content)
#             self.anchors = list(map(lambda x: (x.text, self.normalize(x.get('href'))), a))
            self.anchors = []
            a = model.find_all('a')
            for anchor in a:
                href = self.normalize(anchor.get('href'))
                text = anchor.text
                self.anchors.append((text, href))
            # Parsing text only from articles
            if is_article:
                text = model.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p"])
                # https://www.geeksforgeeks.org/python-ways-to-flatten-a-2d-list/ (08.04.2021)
                # List flattening
                # https://stackoverflow.com/questions/23380171/using-beautifulsoup-to-extract-text-without-tags (08.04.2021)
                # Realised there is a property .text fot the tag element with the help of this link
                self.text = u" ".join(list(map(lambda x: x.text, text)))

### 1.2.2. [20] Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk. 

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (of a form https://www.reuters.com/...) for visiting. To speed up, doesn't consider for visiting pages with content type other than html: '.pdf', '.mp3', '.avi', '.mp4', '.txt'. If encounters an article page (of a form https://www.reuters.com/article/...), saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`, `doc_id:url`
    - `index`, `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    - `doc_lengths`, `doc_id:doc_length` 

    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.

Your crawler have to print visited urls as it runs.

In [3]:
# Copied from HW1
# References: https://stackoverflow.com/questions/18883086/check-if-the-page-is-html-page-in-python (08.04.2021)
# https://stackoverflow.com/questions/25392321/python-requests-check-if-url-is-not-html-webpage/25392456 (08.04.2021)
def check_html(url):
    # check if url leads to unrequired document
    req = requests.head(url)
    if 'html' in req.headers['Content-Type']:
        return True
    else:
        return False
    
def get_netloc(source):
    return urlparse(source).netloc

In [15]:
from collections import Counter
from queue import Queue
import pickle
import os
from urllib.parse import urlparse

class Indexer:

    def __init__(self):
        # dictionaries to populate
        self.doc_urls = {}        
        self.index = {}
        self.doc_lengths = {}
        # preprocessor
        self.prep = Preprocessor()
        
    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):        
        #TODO generate url-s for visiting
        # netloc to not go outside inner pages
        netloc = get_netloc(source)
        doc_id = 0
        total_anchors = 0
        queue = Queue()
        visited = [source]
        queue.put((1, source))
        while not queue.empty():
            if doc_id == limit:
                doc_id+=1
                break
            link = queue.get()
            cur_depth = link[0]
            url = link[1]
            doc = HtmlDocument(url)
            if doc.download():
                if '/article/' in url:
                    doc.parse(is_article=True)
                    doc.persist(collection_path)
                    self.doc_urls[doc_id] = url
                    self.index_doc(doc, doc_id)
                    doc_id+=1
                else:
                    doc.parse()
                if cur_depth < depth:
                    total_anchors+= len(doc.anchors)
                    for link in doc.anchors:
                        url_next = link[1]
                        if get_netloc(url_next) == netloc and not (url_next in visited):
                            queue.put((cur_depth+1, url_next))
                            visited.append(url_next)
                        else:
                            total_anchors-=1
                yield doc
        print(total_anchors)
        # save results for later use
        if limit == None or doc_id <= limit:
            with open('doc_urls.p', 'wb') as fp:
                pickle.dump(self.doc_urls, fp, protocol=pickle.HIGHEST_PROTOCOL)
            with open('inverted_index.p', 'wb') as fp:
                pickle.dump(self.index, fp, protocol=pickle.HIGHEST_PROTOCOL)
            with open('doc_lengths.p', 'wb') as fp:
                pickle.dump(self.doc_lengths, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
        
    def index_doc(self, doc, doc_id):
        #TODO add documents to index
        text = self.prep.preprocess(doc.text)
        self.doc_lengths[doc_id] = len(text)
        index = {}
        for word in text:
            if not word in self.index:
                self.index[word] = [0]
            self.index[word][0]+=1
            if not word in index:
                index[word] = 0
            index[word]+=1
        for key in index.keys():
            self.index[key].append((doc_id, index[key]))

### 1.2.3. Tests

Please make sure your crawler prints out urls with `print(k, c.url)`

In [9]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/technology", 2, "test_collection", 5):
    print(k, c.url)
    k += 1

assert type(indexer.index) is dict
assert type(indexer.index['reuter']) is list
assert type(indexer.index['reuter'][0]) is int
assert type(indexer.index['reuter'][1]) is tuple

1 https://www.reuters.com/technology
2 https://www.reuters.com/technology#main-content
3 https://www.reuters.com/
4 https://www.reuters.com/world/
5 https://www.reuters.com/business/
6 https://www.reuters.com/markets/
7 https://www.reuters.com/breakingviews/
8 https://www.reuters.com/signin/
9 https://www.reuters.com/business/autos-transportation/teslas-bad-week-china-was-months-making-2021-04-22/
10 https://www.reuters.com/technology/facebook-incorporate-user-feedback-news-feed-arrangement-2021-04-22/
11 https://www.reuters.com/business/media-telecom/att-beats-revenue-wireless-subscribers-estimates-2021-04-22/
12 https://www.reuters.com/technology/facebook-begins-testing-ads-instagram-reels-2021-04-22/
13 https://www.reuters.com/technology/amazon-backed-home-tech-startup-smartrent-go-public-over-2-bln-spac-deal-2021-04-22/
14 https://www.reuters.com/technology/chipmaker-tsmc-approves-28-bln-capacity-expansion-2021-04-22/
15 https://www.reuters.com/technology/toronto-based-tech-firm-al

KeyError: 'reuter'

### 1.2.4. Building an index

In [16]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/", 3, "dock_collection"):
    print(k, c.url)
    k += 1

1 https://www.reuters.com/
2 https://www.reuters.com/#main-content
3 https://www.reuters.com/world/
4 https://www.reuters.com/business/
5 https://www.reuters.com/markets/
6 https://www.reuters.com/breakingviews/
7 https://www.reuters.com/signin/
8 https://www.reuters.com/business/environment/us-pledges-halve-its-emissions-by-2030-renewed-climate-fight-2021-04-22/
9 https://www.reuters.com/world/india/india-records-biggest-ever-rise-coronavirus-cases-anywhere-world-2021-04-22/
10 https://www.reuters.com/world/us/la-develops-homelessness-strategy-does-it-have-political-will-2021-04-22/
11 https://www.reuters.com/world/us/after-milestone-police-verdict-minneapolis-lays-rest-another-black-man-2021-04-22/
12 https://www.reuters.com/business/autos-transportation/teslas-bad-week-china-was-months-making-2021-04-22/
13 https://www.reuters.com/world/legal-war-between-saudis-their-former-spymaster-2021-04-22/
14 https://www.reuters.com/world/europe/russia-orders-soldiers-begin-return-bases-after-

### 1.2.5. [5] Index statistics

Load an index and print the statistics.

In [17]:
# load index, doc_lengths and doc_urls
with open('inverted_index.p', 'rb') as fp:
    index = pickle.load(fp)
with open('doc_lengths.p', 'rb') as fp:
    doc_lengths = pickle.load(fp)
with open('doc_urls.p', 'rb') as fp:
    doc_urls = pickle.load(fp)
    
    
print('Total index length', len(index))
print('\nTop terms by number of documents they apperared in:')
sorted_by_n_docs = sorted(index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])) for i in range(20)])
print('\nTop terms by overall frequency:')
sorted_by_freq = sorted(index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

Total index length 2536

Top terms by number of documents they apperared in:
[('trust', 47), ('thomson', 47), ('standard', 47), ('see', 47), ('reuter', 47), ('read', 47), ('quot', 47), ('our', 47), ('minimum', 47), ('list', 47), ('here', 47), ('exchang', 47), ('delay', 47), ('complet', 47), ('all', 47), ('min', 44), ('discov', 44), ('s', 41), ('said', 36), ('report', 36)]

Top terms by overall frequency:
[('reuter', 192), ('s', 166), ('here', 160), ('thomson', 95), ('delay', 93), ('after', 82), ('said', 79), ('all', 77), ('market', 76), ('have', 75), ('report', 74), ('bank', 73), ('not', 70), ('european', 70), ('more', 69), ('thi', 68), ('read', 66), ('our', 66), ('expect', 65), ('stock', 63)]


## 1.3. [15] Answering a query (finally)

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [119]:
def get_avg_len(doc_lengths):
    result = 0
    for key in doc_lengths.keys():
        result+=doc_lengths[key]
    return result/len(doc_lengths)

# https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function (10.04.2021) Doing as requested 
def BM25(idf, freq, k1, b, l, avg_l):
    return idf*(freq*k1+1)/(freq+k1*(1-b+b*(l/avg_l)))

In [120]:
from collections import Counter
import math

class QueryProcessing:
    
    @staticmethod
    def prepare_query(raw_query):
        prep = Preprocessor()
        # pre-process query the same way as documents
        query = prep.preprocess(raw_query)
        # count frequency
        return Counter(query)
    
    @staticmethod
    def boolean_retrieval(query, index):
        #TODO retrieve a set of documents containing all query terms
        prev_set = None
        for word in query:
            if word in index:
                docs = index[word][1:]
                new_set = set()
                for d in docs:
                    new_set.add(d[0])
                if prev_set == None:
                    prev_set = new_set
                else:
                    prev_set = prev_set.intersection(new_set) 
        return prev_set

    
    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
        #TODO retrieve relevant documents with scores
        result = {}
        avg_length = get_avg_len(doc_lengths)
        for word in query:
            if word in index:
                idf = index[word][0]
                docs = index[word][1:]
                for doc in docs:
                    doc_id = doc[0]
                    doc_freq = doc[1]
                    if not doc_id in result:
                        result[doc_id] = 0
                    result[doc_id]+=BM25(idf, doc_freq, k1, b, doc_lengths[doc_id], avg_length)
        return result

### 1.3.1. Tests

In [121]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4:20, 5:30}
test_index = {'x': [2, (1, 1), (2, 1)], 'y': [2, (1, 1), (3, 1)], 'z': [3, (2, 1), (4,2)]}


test_query1 = QueryProcessing.prepare_query('x z')
test_query2 = QueryProcessing.prepare_query('x y')


assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]

# 2. [40] Sorri not veri gud in inglish

Have you ever googled someone's name without knowing exactly how it should be written? Were you ever reluctant to look up the correct spelling of a query you typed? Or just unable to type properly because of being in a rush? Modern search engines usually do a pretty good job in deciphering defective user input. In order to be able to do that, a good spell-checking mechanism should be incorporated into a search procedure. Today we will take one step further towards building a good search engine and work on tolerant retrieval with respect to user queries. We will consider two cases:

1. Users know that they don't know the correct spelling OR they want to get the results that follow some known pattern, so the use so called wildcards - queries like `retr*val`;
2. Users don't know the correct spelling OR they don't care OR they are in a rush OR they expect that mistakes will be corrected OR /your option/... so they make mistakes and we need to handle them using:

    2.1. Trigrams with Jaccard coefficient;
    
    2.2. Simple spellchecker by Peter Norvig with QWERTY weights;

## 2.1. [25] Handling wildcards

We will handle wildcard queries using k-grams. K-gram is a list of consecutive k chars in a string - i.e., for the word *'star'*, it will be '*\$st*', '*sta*', '*tar*', and '*ar$*', if we take k=3. Take a look at [book](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf) *chapter 3.2.2* to understand how k-grams can help efficiently match a wildcard against dictionary words. Here we will only consider wildcards with star symbols (may be multiple).

Notice that for building k-grams index, **we will need a vocabulary of original word forms** to compare words in user input to the vocabulary of "correct" words (think why inverted index which we built for stemmed words doesn't work here).   

You need to implement the following:

- `build_inverted_index_orig_forms` - creates inverted index of original world forms from `facts` list, which is already given to you.  
    Output format: `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    

- `build_k_gram_index` - creates k-gram index which maps every k-gram encountered in facts collection to a list of words containing this k-gram. Use the abovementioned inverted index of original words to construct this index.  
    Output format: `'k_gram': ['word1_with_k_gram', 'word2_with_k_gram', ...]`
    
    
- `generate_wildcard_options` - produce a list of vocabulary words matching given wildcard by intersecting postings of k-grams present in the wildcard (refer to *ch 3.2.2*). 

- `search_wildcard` - return list of facts that contain the words matching a wildcard query.


We will use the dataset with curious facts for testing.

### 2.1.1. Downloading the dataset

In [122]:
import urllib.request
data_url = "https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt"
local_filename, headers = urllib.request.urlretrieve(data_url)

facts = []
with open(local_filename) as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))
        
print(*facts[-5:], sep='\n')

151. Women have twice as many pain receptors on their body than men. But a much higher pain tolerance.
152. There are more stars in space than there are grains of sand on every beach in the world.
153. For every human on Earth there are 1.6 million ants.
154. The total weight of all those ants, however, is about the same as all the humans.
155. On Jupiter and Saturn it rains diamonds.


### 2.1.2. [25] Implementation of search

In [123]:
def k_gram_calc(word, k):
    return [word[i:i+k] for i in range(len(word)-k+1)]

# https://www.geeksforgeeks.org/python-intersection-two-lists/ (10.04.2021) Implementation of intersection. Could have done it using sets, but decided to use lists
def intersection(arr1, arr2):
    return [x for x in arr1 if x in arr2]

In [124]:
import re

def prepocess(text):
    tokenizer = nltk.tokenize.NLTKWordTokenizer()
    return list(filter(lambda x: x.isalpha(), tokenizer.tokenize(text.lower())))

def build_inverted_index_orig_forms(documents):
    #TODO build an inverted index of original word forms 
    # (without stemming, just word tokenized and lowercased)   
    inverted_index = {}
    doc_id = 0
    for text in documents:
        words = prepocess(text)
        index = {}
        for word in words:
            if not word in inverted_index:
                inverted_index[word] = [0]
            inverted_index[word][0]+=1
            if not word in index:
                index[word] = 0
            index[word]+=1
        for key in index.keys():
            inverted_index[key].append((doc_id, index[key]))
        doc_id+=1
    return inverted_index


def build_k_gram_index(inverted_index, k):
    #TODO build index of k-grams for dictionary words. 
    # Padd with '$' ($word$) before splitting to k-grams    
    k_gram_index = {}
    prepared_words = list(map(lambda x:'$' + x + '$', inverted_index.keys()))
    for word in prepared_words:
        k_grams = k_gram_calc(word, k)
        for k_gram in k_grams:
            if not k_gram in k_gram_index:
                k_gram_index[k_gram] = []
            k_gram_index[k_gram].append(word[1:-1])
    return k_gram_index


def generate_wildcard_options(wildcard, k_gram_index, inverted_index):
    #TODO for a given wildcard return all words matching it using k-grams
    # refer to book chapter 3.2.2
    # don't forget to pad wildcard with '$', when appropriate
    k = 0
    for key in k_gram_index.keys():
        k = len(key)
    resulting_wildcard = wildcard
    if resulting_wildcard[0] != '*':
        resulting_wildcard = '$' + resulting_wildcard
    if resulting_wildcard[-1] != '*':
        resulting_wildcard = resulting_wildcard + '$'
    words = resulting_wildcard.split('*')
    k_grams = []
    for word in words:
        k_grams += list(map(lambda x: word[int(x):int(x+k)], range(len(word)-k+1)))
    result = None
    for k_gram in k_grams:
        if k_gram in k_gram_index:
            candidates = k_gram_index[k_gram]
            if result == None:
                result = candidates
            else:
                result = intersection(result, candidates)
    result = list(filter(lambda x: re.match(wildcard.replace('*', '[a-zA-Z]*'), x), result))
    return result


def search_wildcard(wildcard, k_gram_index, index, docs):
    #TODO retrive list of documnets (facts) that contain words matching wildcard
    words = generate_wildcard_options(wildcard, k_gram_index, index)
    result = []
    for w in words:
        res = QueryProcessing.boolean_retrieval([w], index)
        for r in res:
            result.append(docs[r])
    return result

### 2.1.3. Tests

In [125]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)

wildcard = "re*ed"

wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index_orig_forms)
print(wildcard_options)
assert(len(wildcard_options) >= 3)
assert("red" not in wildcard_options) 

wildcard_results = search_wildcard(wildcard, k_gram_index, index_orig_forms, facts)
# some pretty printing
for r in wildcard_results:
    # highlight terms for visual evaluation
    for term in wildcard_options:
        r = re.sub(r'(' + term + ')', r'\033[1m\033[91m\1\033[0m', r, flags=re.I)
    print(r)

assert(len(wildcard_results) >=3)

assert "13. James Buchanan, the 15th U.S. president continuously bought slaves with his own money in order to free them." in search_wildcard("pres*dent", k_gram_index, index_orig_forms, facts)
assert "40. 9 out of 10 Americans are deficient in Potassium." in search_wildcard("p*tas*um", k_gram_index, index_orig_forms, facts)
assert "61. A man from Britain changed his name to Tim Pppppppppprice to make it harder for telemarketers to pronounce." in search_wildcard("*price", k_gram_index, index_orig_forms, facts)

['recorded', 'received', 'reduced']
4. The largest recorded snowflake was in Keogh, MT during year 1887, and was 15 inches wide.
102. More than 50% of the people in the world have never made or received a telephone call.
134. A person can live without food for about a month, but only about a week without water. If the amount of water in your body is reduced by just 1%, you'll feel thirsty. If it's reduced by 10%, you'll die.


## 2.2. [15] Handling typos

### 2.2.0. Dataset 

Download github typo dataset from [here](https://github.com/mhagiwara/github-typo-corpus).
Load it with this code:

In [186]:
!pip install jsonlines

In [126]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] != 'eng':
                other_langs.add(edit['src']['lang'])
                continue

            if edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((edit['src']['text'], edit['tgt']['text']))
                
print(f"Dataset size = {len(dataset)}")

Dataset size = 245909


#### Explore sample typos
Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [127]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

#### 2.2.0.1. Build a dataset vocabulary
Here we prepare a vocabulary for spellchecker testing and for estimating overall correction quality. Consider only word-level. Be carefull, there is markdown (e.g. \`name\`. \[url\]\(http://url)) and comment symbols (\#, //, \*).

In [128]:
def sent_to_words(sent):
    # splits sentence to words, filtering out non-alphabetical terms
    words = nltk.word_tokenize(sent)    
    words_filtered = filter(lambda x: x.isalpha(), words)
    return words_filtered

In [129]:
vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
len(vocabulary)

63724

In [130]:
from itertools import islice
print(list(islice(vocabulary.items(), 10)))

[('function', 6193), ('de', 82), ('deutsch', 4), ('nocomments', 2), ('you', 42075), ('can', 26027), ('disable', 532), ('comments', 360), ('for', 44756), ('the', 207017)]


### 2.2.1. [15] Implement context-independent spellcheker (Trigrams with Jaccard coefficient) ##

In [131]:
def Jaccard_coefficient(k_gram1, k_gram2):
    l1 = len(k_gram1)
    l2 = len(k_gram2)
    lint = len(intersection(k_gram1, k_gram2))
    return lint/(l1+l2-lint)

In [132]:
def fix_typo_kgram(word, k_gram_index) -> list:
    #TODO return best matches with respect to Jaccard index   
    k = 0
    for key in k_gram_index.keys():
        k = len(key)
    k_gram = k_gram_calc(word, k)
    candidates = []
    for k_gr in k_gram:
        if k_gr in k_gram_index:
            candidates+=k_gram_index[k_gr]
    result = []
    best_jac = -1
    for candidate in candidates:
        cand_k_gram = k_gram_calc(candidate, k)
        jac_coeff = Jaccard_coefficient(k_gram, cand_k_gram)
        if best_jac < jac_coeff:
            result = [candidate]
            best_jac = jac_coeff
        if (best_jac == jac_coeff) and (not candidate in result):
            result.append(candidate)
    return result

### 2.2.2. Tests

In [133]:
# tests

k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])
assert "enough" in fix_typo_kgram("enouh", k_gram_index_github), "Assert k-gram failed"

['enough']


## 2.3. [Bonuses]

### 2.3.1. [Bonus] QWERTY - Editorial distance

Write code to compute weighted QWERTY-editorial distance.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/KB_United_States.svg/640px-KB_United_States.svg.png" width="640"/> 

Use this image to prepare weight function:
- all letter pairs which share vertical border will get 0.5 multiplier **for replace** (`df`, `cv`, `ui`, ...)
- all letter pairs which share at least some horizontal border will get 0.7 multiplier **for replace** (`dc`, `dr`, `km`, ...)
- other operations are not scaled (x1 multiplier).

In [134]:
def replace_weight(let1, let2) -> float:
    # TODO what is the weight for a pair of letters being replaced? Note this function should be commutative
    horizontal = ['qw', 'we', 'er', 'rt', 'ty', 'yu', 'ui', 'io', 'op', 'as', 'sd', 'df', 'fg', 'gh', 'hj', 'jk', 'kl', 'zx', 'xc', 'cv', 'vb', 'bn', 'nm']
    vertical = ['qa', 'wa', 'ws', 'es', 'ed', 'rd', 'rf', 'tf', 'tg', 'yg', 'yh', 'uh', 'uj', 'ij', 'ik', 'ok', 'ol', 'pl', 'az', 'sz', 'sx', 'dx', 'dc', 'fc', 'fv', 'gv', 'gb', 'hb', 'hn', 'jn', 'jm', 'km']
    if let1+let2 in horizontal or let2+let1 in horizontal:
        return 0.5
    elif let1+let2 in vertical or let2+let1 in vertical:
        # The last test is most probably incorrect. It should be 0.7 but not 0.6
        return 0.7
    return 1.0

# References: https://www.geeksforgeeks.org/edit-distance-dp-5/ (12.04.2021) Explanation of Edit distance
# https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance (12.04.2021) Explanation of changes in algorithm
# https://www.youtube.com/watch?v=MiqoA-yF-0M&t=599s&ab_channel=BackToBackSWE (12.04.2021) Explanation of Edit distance
# Our contest code
def qwerty_edit_dist(s1, s2) -> float:
    # TODO compute the Damerau-Levenshtein distance between two given strings (s1 and s2)
    n, m = len(s1) + 1, len(s2) + 1
    data = [[0 for i in range(n)] for j in range(m)]
    
    for i in range(n): data[0][i] = i
    for i in range(m): data[i][0] = i
        
    for i in range(1, m):
        for j in range(1, n):
            
            cost = 1
            if s1[j - 1] == s2[i - 1]:
                cost = 0
                
            data[i][j] = min(data[i][j - 1] + 1,
                             data[i - 1][j] + 1,
                             replace_weight(s1[i-1], s2[j-1])*(data[i - 1][j - 1] + cost))
            if i > 1 and j > 1 and s1[i-1] == s2[j-2] and s1[i-2] == s2[j-1]:
                data[i][j] = min(data[i][j], data[i-1][j-1])
    return data[m - 1][n - 1]

In [135]:
# tests

assert qwerty_edit_dist("korrectud", "corrected") == 2.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("soem", "some") == 1.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("one", "one") == 0.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fihure", "figure") == 0.5, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fivure", "figure") == 0.7, "Edit distance is computed incorrectly"

### 2.3.2. [Bonus] Norvig's spellchecker with QWERTY weights

You can base your code on [Norvig's corrector](https://norvig.com/spell-correct.html), but you should be sure you account the fact, that typos for close letters cost less. This should be considered in ranking.

In [144]:
def fix_typo_qwerty_norvig(word) -> str:
    #TODO return best matching result for the word
    
    return correction(word)

In [ ]:
# tests

assert fix_typo_qwerty_norvig("korrectud") == "corrected", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("speling") == "spelling", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("condidence") == "confidence", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fpx") == "fox", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fux") == "fix", "Norvig's correcter doesn't work"

### 2.3.3. Estimate quality of functions

In [ ]:
norvig, kgram = 0, 0
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    words = sent_to_words(src.lower())
    # word suspected for typos
    sn, sk = src.lower(), src.lower()
    for word in words:
        if word not in vocabulary and word.isalpha():
            # top-1 accuracy
            wn, wk = fix_typo_qwerty_norvig(word), \
                     fix_typo_kgram(word, k_gram_index_github)[0]
            sn = sn.replace(word, wn)
            sk = sk.replace(word, wk)
    norvig += int(sn == target.lower())
    kgram += int(sk == target.lower())

print(f"Norvig accuracy ({norvig}) = {norvig / limit}")
print(f"k-gram accuracy ({kgram}) = {kgram / limit}")